# Download, pick, and generate data of specific genomic contexts.
To pick the interested region of specific genomic context and generate datasets of them, an example of GRCm39/mm39 on chromosome [10] is performed (Original build in the paper is Ensembl build 104).

There are three types of raw files:
- Regulatory Features: promoter, enhancers, TFBS, open chromatin, CTCF BS
- Featuer types: Exons, CDS
- Histone binding sites: H3K27ac, H3K27me3, H3K36me3, H3K4me1, H3K4me3, H3K9ac, H3K9me3, p300
- CpG featuers: CpG islands, CpG shores

Three steps are included respectively:
- Download 
- Pick interested regions and corresponding index
- Generate dataset by new index

In [71]:
import numpy as np
import os
import requests
import gzip
import shutil
import pandas as pd
import bisect
import subprocess
from tqdm import tqdm
# set chromosome ID 
target_chromosome = ['10'] #chromosome 10 for example
# load files of Ser Dataset
y = np.load("y.npz")# your y.npz file path of Ser dataset
pos = np.load("pos.npz")# your pos.npz file path of Ser dataset

# Regulatory Features

Available regions: {'promoter', 'CTCF_binding_site', 'open_chromatin_region', 'enhancer', 'TF_binding_site'}

## Download

In [5]:
# Define the URL and filename
url = 'https://ftp.ensembl.org/pub/release-109/regulation/mus_musculus/mus_musculus.GRCm39.Regulatory_Build.regulatory_features.20221007.gff.gz'
filename = 'mus_musculus.GRCm39.Regulatory_Build.regulatory_features.20221007.gff.gz'
unzip_filename = 'mus_musculus.GRCm39.Regulatory_Build.regulatory_features.20221007.gff'

if not os.path.exists(unzip_filename):
    # Download the file to the current directory
    response = requests.get(url)

    if response.status_code == 200:
        with open(os.path.join(os.getcwd(), filename), 'wb') as f:
            f.write(response.content)

    # Unzip the compressed file
    with gzip.open(filename, 'rb') as f_in:
        with open(unzip_filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

## Pick interested regions and generate the corresponding dataset

In [8]:
# Pick feature types you are interested in
picked_type_list = ['promoter', 'CTCF_binding_site', 'open_chromatin_region', 'enhancer', 'TF_binding_site']
# Load GFF file into pandas DataFrame
df = pd.read_csv(unzip_filename, sep='\t', header=None, usecols=range(5))
column_names = ['chr_id', 'Regulatory_build', 'type', 'start', 'end']
df.columns = column_names
for pick_type_id in picked_type_list:
    print('Picked type: ', pick_type_id)
    pick_type = pick_type_id
    if not os.path.exists('y_'+pick_type+'.npz'):
        y_picked = {}
        pos_picked = {}
        for target_chromosome_id in tqdm(target_chromosome):
            # Select specific chromosome
            df_chr = df[df['chr_id'] == target_chromosome_id][['type', 'start', 'end']]
            # Available elements
            # print("Available regions:",set(df_chr['type']))
            # Take promoter as an example
            df_chr_pick = df_chr[df_chr['type'] == pick_type][['start', 'end']]
            df_chr_pick = df_chr_pick.sort_values(by='start').reset_index(drop=True)
            chr_pick = df_chr_pick.values

            chromosome_name = 'chr'+target_chromosome_id
            # select chromosome
            y_chr = y[chromosome_name]
            pos_chr = pos[chromosome_name]
            # initial new dataset
            y_chr_picked = []
            pos_chr_picked = [] 
            [last_start, last_end] = [0, 0]
            for i in range(chr_pick.shape[0]):
                [start, end] = chr_pick[i, :]
                if last_start == start or start < last_end:
                    continue   
                start_pick = bisect.bisect_left(pos_chr, start)
                end_pick = bisect.bisect_right(pos_chr, end) - 1
                if end_pick <= 0:
                    continue
                if end_pick > pos_chr.shape[0]:
                    continue
                y_chr_picked += list(y_chr[start_pick:end_pick,:])
                pos_chr_picked += list(pos_chr[start_pick:end_pick])
                [last_start, last_end] = [start, end]
            y_chr_picked = np.array(y_chr_picked)

            y_picked[chromosome_name] = y_chr_picked
            pos_picked[chromosome_name] = pos_chr_picked
            
        # save as npz
        np.savez_compressed('y_'+pick_type+'.npz', **y_picked)
        np.savez_compressed('pos_'+pick_type+'.npz', **pos_picked)

Picked type:  promoter


100%|██████████| 1/1 [00:00<00:00,  7.87it/s]


Picked type:  CTCF_binding_site


100%|██████████| 1/1 [00:00<00:00,  7.95it/s]


Picked type:  open_chromatin_region


100%|██████████| 1/1 [00:00<00:00,  7.85it/s]


Picked type:  enhancer


100%|██████████| 1/1 [00:00<00:00,  6.13it/s]


Picked type:  TF_binding_site


100%|██████████| 1/1 [00:00<00:00,  8.42it/s]


# Feature types

Available regions: {'ncRNA', 'CDS', 'unconfirmed_transcript', 'transcript', 'ncRNA_gene', 'mRNA', 'scRNA', 'biological_region', 'five_prime_UTR', 'rRNA', 'snoRNA', 'pseudogene', 'snRNA', 'miRNA', 'lnc_RNA', 'pseudogenic_transcript', 'exon', 'three_prime_UTR', 'chromosome', 'gene'}

### Download

In [10]:
for target_chromosome_id in tqdm(target_chromosome):
    # Define the URL and filename
    url = 'https://ftp.ensembl.org/pub/release-109/gff3/mus_musculus/Mus_musculus.GRCm39.109.chromosome.'+target_chromosome_id+'.gff3.gz'
    filename = 'Mus_musculus.GRCm39.109.chromosome.'+target_chromosome_id+'.gff3.gz'
    unzip_filename = 'Mus_musculus.GRCm39.109.chromosome.'+target_chromosome_id+'.gff3'

    if not os.path.exists(unzip_filename):
        # Download the file to the current directory
        response = requests.get(url)

        if response.status_code == 200:
            with open(os.path.join(os.getcwd(), filename), 'wb') as f:
                f.write(response.content)

        # Unzip the compressed file
        with gzip.open(filename, 'rb') as f_in:
            with open(unzip_filename, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

100%|██████████| 1/1 [00:09<00:00,  9.47s/it]


## Pick interested regions and generate the corresponding dataset

In [12]:
# Pick feature types you are interested in
picked_type_list = ['CDS', 'exon']
for pick_type_id in ['CDS', 'exon']:
    print('Picked type: ', picked_type_list)
    pick_type = pick_type_id
    if not os.path.exists('y_'+pick_type+'.npz'):
        y_picked = {}
        pos_picked = {}
        for target_chromosome_id in tqdm(target_chromosome[:]):
            # Load GFF file into pandas DataFrame
            df = pd.read_csv('Mus_musculus.GRCm39.109.chromosome.'+target_chromosome_id+'.gff3', sep='\t', header=None, comment='#', usecols=range(2,5))
            column_names = ['feature_type', 'start', 'end']
            df.columns = column_names
            # Available features
            # print("Available regions:",set(df['feature_type']))

            # Take exons as an example
            df_chr_pick = df[df['feature_type'] == pick_type][['start', 'end']]
            df_chr_pick = df_chr_pick.sort_values(by='start').reset_index(drop=True)
            chr_pick = df_chr_pick.values

            # load files of Ser Dataset
            chromosome_name = 'chr'+target_chromosome_id
            # select chromosome
            y_chr = y[chromosome_name]
            pos_chr = pos[chromosome_name]
            # initial new dataset
            y_chr_picked = []
            pos_chr_picked = [] 
            [last_start, last_end] = [0, 0]
            for i in range(chr_pick.shape[0]):
                [start, end] = chr_pick[i, :]
                if last_start == start or start < last_end:
                    continue    
                start_pick = bisect.bisect_left(pos_chr, start)
                end_pick = bisect.bisect_right(pos_chr, end) - 1
                if end_pick <= 0:
                    continue
                if end_pick > pos_chr.shape[0]:
                    continue
                y_chr_picked += list(y_chr[start_pick:end_pick,:])
                pos_chr_picked += list(pos_chr[start_pick:end_pick])
                [last_start, last_end] = [start, end]
            y_chr_picked = np.array(y_chr_picked)

            y_picked[chromosome_name] = y_chr_picked
            pos_picked[chromosome_name] = pos_chr_picked
        # save as npz
        np.savez_compressed('y_'+pick_type+'.npz', **y_picked)
        np.savez_compressed('pos_'+pick_type+'.npz', **pos_picked)

Picked type:  CDS


100%|██████████| 1/1 [00:00<00:00,  4.92it/s]


Picked type:  exon


100%|██████████| 1/1 [00:00<00:00,  4.30it/s]


# Histone binding sites

Available regions: {'H3K27ac', 'H3K27me3', 'H3K36me3', 'H3K4me1', 'H3K4me3', 'H3K9ac', 'H3K9me3', 'EP300'}

## Download

In [43]:
download_histone_list = ['H3K27ac', 'H3K36me3', 'H3K4me1', 'H3K4me3', 'H3K9ac', 'H3K9me3', 'EP300']
for download_histone_id in tqdm(download_histone_list):
    # Define the URL and filename
    if download_histone_id in ['H3K27ac', 'H3K9ac', 'H3K4me3', 'EP300']:
        url = 'https://ftp.ensembl.org/pub/release-109/regulation/mus_musculus/Peaks/ES_Bruce4_embryonic/'+download_histone_id+'/mus_musculus.GRCm39.ES_Bruce4_embryonic.'+download_histone_id+'.SWEmbl_R0005.peaks.20201003.bed.gz'
        filename = 'mus_musculus.GRCm39.ES_Bruce4_embryonic.'+download_histone_id+'.SWEmbl_R0005.peaks.20201003.bed.gz'
        unzip_filename = 'mus_musculus.GRCm39.ES_Bruce4_embryonic.'+download_histone_id+'.SWEmbl_R0005.peaks.20201003.bed'
    else:
        url = 'https://ftp.ensembl.org/pub/release-109/regulation/mus_musculus/Peaks/ES_Bruce4_embryonic/'+download_histone_id+'/mus_musculus.GRCm39.ES_Bruce4_embryonic.'+download_histone_id+'.ccat_histone.peaks.20201003.bed.gz'
        filename = 'mus_musculus.GRCm39.ES_Bruce4_embryonic.'+download_histone_id+'.ccat_histone.peaks.20201003.bed.gz'
        unzip_filename = 'mus_musculus.GRCm39.ES_Bruce4_embryonic.'+download_histone_id+'.ccat_histone.peaks.20201003.bed'
    
    response = requests.get(url)

    if response.status_code == 200 and (not os.path.exists(unzip_filename)):
        with open(os.path.join(os.getcwd(), filename), 'wb') as f:
            f.write(response.content)

        # Unzip the compressed file
        with gzip.open(filename, 'rb') as f_in:
            with open(unzip_filename, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

100%|██████████| 7/7 [00:37<00:00,  5.42s/it]


## Pick interested regions and generate the corresponding dataset

In [72]:
# Pick feature types you are interested in
picked_type_list = ['H3K27ac', 'H3K36me3', 'H3K4me1', 'H3K4me3', 'H3K9ac', 'H3K9me3', 'EP300']
for pick_type_id in tqdm(picked_type_list):
    print('Picked type: ', pick_type_id)
    pick_type = pick_type_id
    if pick_type_id in ['H3K27ac', 'H3K9ac', 'H3K4me3', 'EP300']:
        unzip_filename = 'mus_musculus.GRCm39.ES_Bruce4_embryonic.'+pick_type_id+'.SWEmbl_R0005.peaks.20201003.bed'
    else:
        unzip_filename = 'mus_musculus.GRCm39.ES_Bruce4_embryonic.'+pick_type_id+'.ccat_histone.peaks.20201003.bed'
    if not os.path.exists('y_'+pick_type+'.npz'):
        # Load bed file into pandas DataFrame
        df = pd.read_csv(unzip_filename, sep='\t', header=None, usecols=range(3))
        column_names = ['chr_id', 'start', 'end']
        df.columns = column_names

        y_picked = {}
        pos_picked = {}
        for target_chromosome_id in tqdm(target_chromosome):
            chromosome_name = 'chr'+target_chromosome_id
            # Select specific chromosome
            df_chr_pick = df[df['chr_id'] == chromosome_name][['start', 'end']]
            df_chr_pick = df_chr_pick.sort_values(by='start').reset_index(drop=True)
            chr_pick = df_chr_pick.values
            # select chromosome
            y_chr = y[chromosome_name]
            pos_chr = pos[chromosome_name]
            # initial new dataset
            y_chr_picked = []
            pos_chr_picked = [] 
            [last_start, last_end] = [0, 0]
            for i in range(chr_pick.shape[0]):
                [start, end] = chr_pick[i, :]
                if last_start == start or start < last_end:
                    continue   
                start_pick = bisect.bisect_left(pos_chr, start)
                end_pick = bisect.bisect_right(pos_chr, end) - 1
                if end_pick <= 0:
                    continue
                if end_pick > pos_chr.shape[0]:
                    continue
                y_chr_picked += list(y_chr[start_pick:end_pick,:])
                pos_chr_picked += list(pos_chr[start_pick:end_pick])
                [last_start, last_end] = [start, end]
            y_chr_picked = np.array(y_chr_picked)

            y_picked[chromosome_name] = y_chr_picked
            pos_picked[chromosome_name] = pos_chr_picked

        # save as npz
        np.savez_compressed('y_'+pick_type+'.npz', **y_picked)
        np.savez_compressed('pos_'+pick_type+'.npz', **pos_picked)

  0%|          | 0/7 [00:00<?, ?it/s]

Picked type:  H3K27ac


100%|██████████| 1/1 [00:00<00:00, 17.50it/s]


Picked type:  H3K36me3


 29%|██▊       | 2/7 [00:00<00:00,  9.92it/s]

...

 71%|███████▏  | 5/7 [00:00<00:00, 10.65it/s]

Picked type:  H3K9me3


100%|██████████| 1/1 [00:00<00:00, 13.76it/s]


Picked type:  EP300


100%|██████████| 7/7 [00:00<00:00, 10.51it/s]


# CpG islands and CpG shores
 
CpG shores: are processed with the regions 0-2000 positions both down and upstream from those CpG Islands

['CpG_island', 'CpG_shores']

## Download

In [13]:
# Define the URL and filename
url = 'http://hgdownload.cse.ucsc.edu/goldenpath/mm39/database/cpgIslandExt.txt.gz'
filename = 'cpgIslandExt.txt.gz'
unzip_filename = 'cpgIslandExt.txt'

if not os.path.exists(unzip_filename):
    # Download the file to the current directory
    response = requests.get(url)

    if response.status_code == 200:
        with open(os.path.join(os.getcwd(), filename), 'wb') as f:
            f.write(response.content)

    # Unzip the compressed file
    with gzip.open(filename, 'rb') as f_in:
        with open(unzip_filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

## Pick interested regions and generate the corresponding dataset

In [14]:
# Pick feature types you are interested in
picked_type_list = ['shores', 'islands']
# Load txt file into pandas DataFrame
df = pd.read_csv('cpgIslandExt.txt', sep='\t', header=None, usecols=range(1,4))
column_names = ['chr_id', 'start', 'end']
df.columns = column_names

for pick_type_id in picked_type_list:
    print('Picked type: ', pick_type_id)
    pick_type = pick_type_id
    if not os.path.exists('y_CpG_'+pick_type+'.npz'):
        y_picked = {}
        pos_picked = {}
        for target_chromosome_id in tqdm(target_chromosome):
            chromosome_name = 'chr'+target_chromosome_id
            # Select specific chromosome
            df_chr = df[df['chr_id'] == chromosome_name][['start', 'end']]
            df_chr_pick = df_chr.sort_values(by='start').reset_index(drop=True)
            chr_pick = df_chr_pick.values

            # select chromosome
            y_chr = y[chromosome_name]
            pos_chr = pos[chromosome_name]
            # initial new dataset
            y_chr_picked = []
            pos_chr_picked = [] 
            [last_start, last_end] = [0, 0]
            for i in range(chr_pick.shape[0]):
                [start, end] = chr_pick[i, :]
                if pick_type == 'shores': # CpG shores
                    if last_start == start-2000 or start-2000 < last_end:
                        continue  
                    startL_pick = bisect.bisect_left(pos_chr, start-2000)
                    endL_pick = bisect.bisect_right(pos_chr,start) - 1
                    startR_pick = bisect.bisect_left(pos_chr, end)
                    endR_pick = bisect.bisect_right(pos_chr,end+2000) - 1
                    if endR_pick <= 0:
                        continue
                    if endR_pick > pos_chr.shape[0]:
                        continue
                    y_chr_picked += list(y_chr[startL_pick:endL_pick,:])
                    y_chr_picked += list(y_chr[startR_pick:endR_pick,:])
                    pos_chr_picked += list(pos_chr[startL_pick:endL_pick])
                    pos_chr_picked += list(pos_chr[startR_pick:endR_pick])
                    [last_start, last_end] = [start-2000, end+2000]
                else:
                    if last_start == start or start < last_end:
                        continue  
                    start_pick = bisect.bisect_left(pos_chr, start)
                    end_pick = bisect.bisect_right(pos_chr, end) - 1
                    if end_pick <= 0:
                        continue
                    if end_pick > pos_chr.shape[0]:
                        continue
                    y_chr_picked += list(y_chr[start_pick:end_pick,:])
                    pos_chr_picked += list(pos_chr[start_pick:end_pick])
                    [last_start, last_end] = [start, end]
            y_chr_picked = np.array(y_chr_picked)

            y_picked[chromosome_name] = y_chr_picked
            pos_picked[chromosome_name] = pos_chr_picked

        # save as npz
        np.savez_compressed('y_CpG_'+pick_type+'.npz', **y_picked)
        np.savez_compressed('pos_CpG_'+pick_type+'.npz', **pos_picked)
    

Picked type:  shores


100%|██████████| 1/1 [00:00<00:00,  8.50it/s]


Picked type:  islands


100%|██████████| 1/1 [00:00<00:00,  9.20it/s]
